In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from pygini import gini

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
root = r'E:\urban_intensity0801'

In [ ]:
file1 = os.path.join(root, r'output\smod_gte21_point.shp')
df1 = gpd.read_file(file1)#.to_crs(4326)
df1.replace(-9999, 0, inplace=True)

In [ ]:
file2 = os.path.join(root, r'output\GRIP4_Road_ID.csv')
df2 = pd.read_csv(file2)

In [ ]:
for i in range(1,6):
    print(i)
    df2_type = df2.loc[df2['GP_RTP']==i, ['Id','Shape_Length']]
    df2_type_sum = df2_type.groupby('Id').sum().reset_index().rename(columns={'Shape_Length':'Length'+str(i)})
    df2_type_sum.to_csv('road_id_type'+str(i)+'.csv', index=False)

In [ ]:
df2_type1 = pd.read_csv('road_id_type1.csv')
for i in range(2,6):
    df2_type = pd.read_csv('road_id_type'+str(i)+'.csv')
    df2_type1 = pd.merge(df2_type1, df2_type, how='outer')
df2_type1.fillna(0, inplace=True)
df2_type1['Length_total'] = df2_type1.iloc[:,1:].sum(axis=1, numeric_only=True)
df2_type1.to_csv('road_id_total.csv',index=False)

In [ ]:
file3 = os.path.join(root, r'output\GHS_FUA.csv')
df3 = pd.read_csv(file3)

In [ ]:
file4 = os.path.join(root, r'output\FUA_IsSouth.csv')
df4 = pd.read_csv(file4)

In [ ]:
df_merge = pd.merge(pd.merge(df1.rename(columns={'pointid':'Id'}), df2_type1, how='outer'), 
                    df3.loc[:,['eFUA_ID','eFUA_name','Cntry_ISO','Cntry_name']], on='eFUA_ID')
df_merge.fillna(0, inplace=True)

In [ ]:
df_merge['Regions'] = 'North'
df_merge.loc[df_merge['eFUA_ID'].isin(Is_South_eFUA_ID),'Regions'] = 'South'
df_merge['Regions']

In [ ]:
df_cntry1 = df_merge.loc[:,['Cntry_name','Regions']].drop_duplicates()
df_cntry2 = df_merge.loc[:,['Cntry_name','Regions']].drop_duplicates(subset='Cntry_name')

In [ ]:
df_merge.loc[df_merge['Cntry_name']=='Israel', 'Regions'] = 'North'
df_merge.loc[df_merge['Cntry_name']=='Turkey', 'Regions'] = 'South'
df_merge.loc[df_merge['Cntry_name']=='Kazakhstan', 'Regions'] = 'South'
df_merge.loc[df_merge['Cntry_name']=='NorthKorea', 'Regions'] = 'South'

In [ ]:
# check
df_cntry1 = df_merge.loc[:,['Cntry_name','Regions']].drop_duplicates()
df_cntry2 = df_merge.loc[:,['Cntry_name','Regions']].drop_duplicates(subset='Cntry_name')
print(len(df_cntry1), len(df_cntry2))

In [ ]:
df_merge['pop_log'] = np.log(df_merge['Pop']+1)
df_merge['ntl_log'] = np.log(df_merge['LongNTL']+1)

In [ ]:
def norm_col(df, col_name):
    df[col_name+'01'] = (df[col_name]-df[col_name].min())/(df[col_name].max()-df[col_name].min())
    return df

In [ ]:
norm_col(df_merge, 'pop_log')
norm_col(df_merge, 'ntl_log')
norm_col(df_merge, 'Length_total')

norm_col(df_merge, 'BUILT_S')
norm_col(df_merge, 'BUILT_V')
df_merge['built'] = df_merge['BUILT_S01'] + df_merge['BUILT_V01']
norm_col(df_merge, 'built')

In [ ]:
def norm01(df_merge, colname):
    df_merge[colname+'n'] = (df_merge[colname] - df_merge[colname].min())/(df_merge[colname].max() - df_merge[colname].min())

In [ ]:
df_merge['udi01'] = (df_merge['built01']+df_merge['ntl_log01']+df_merge['pop_log01']+df_merge['Length_total01'])/4
norm01(df_merge, 'udi01')

In [ ]:
df_index = df_merge.loc[:,['built01','pop_log01', 'Length_total01', 'ntl_log01']]

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
normalized_data = scaler.fit_transform(df_index)

pca = PCA(n_components=4)
pca.fit(normalized_data)

transformed_data = pca.transform(normalized_data)
explained_variance_ratios = pca.explained_variance_ratio_
print("Transformed Data:\n", transformed_data)
print("\nExplained Variance Ratios:\n", explained_variance_ratios)

In [ ]:
loadings = pca.components_.T / np.sqrt(pca.explained_variance_)
loadings_matrix = pd.DataFrame(loadings, columns=['PC1','PC2','PC3','PC4'], index=df_index.columns)

In [ ]:
num = 3
weight = []
for i in range(0, 4):
    weight.append((loadings[i][:num]*explained_variance_ratios[:num]).sum()/(explained_variance_ratios[:num].sum()))
weight = np.array(weight)
norm_weight = weight/(weight.sum())
pd.DataFrame({'weight':norm_weight}, index=df_index.columns)

In [ ]:
df_merge['udi01w'] = ((np.array(df_index)*norm_weight).T).sum(axis=0)
norm01(df_merge, 'udi01w')

In [ ]:
def norm_col19(df, col_name):
    df[col_name+'19'] = 0.1+((df[col_name]-df[col_name].min())/(df[col_name].max()-df[col_name].min()))*(0.9-0.1)
    return df

In [ ]:
norm_col19(df_merge, 'pop_log')
norm_col19(df_merge, 'ntl_log')
norm_col19(df_merge, 'Length_total')

norm_col19(df_merge, 'BUILT_S')
norm_col19(df_merge, 'BUILT_V')
df_merge['built_mu'] = np.sqrt(df_merge['BUILT_S19'] * df_merge['BUILT_V19'])
norm_col19(df_merge, 'built_mu')

In [ ]:
df_merge['udi19'] = np.power((df_merge['built_mu19']*df_merge['ntl_log19']*df_merge['pop_log19']*df_merge['Length_total19']),1/4)
norm01(df_merge, 'udi19')

In [ ]:
df_index = df_merge.loc[:,['built_mu19','pop_log19', 'Length_total19', 'ntl_log19']]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
normalized_data = scaler.fit_transform(df_index)

pca = PCA(n_components=4)
pca.fit(normalized_data)

transformed_data = pca.transform(normalized_data)
explained_variance_ratios = pca.explained_variance_ratio_
print("Transformed Data:\n", transformed_data)
print("\nExplained Variance Ratios:\n", explained_variance_ratios)

In [ ]:
loadings = pca.components_.T / np.sqrt(pca.explained_variance_)
loadings_matrix = pd.DataFrame(loadings, columns=['PC1','PC2','PC3','PC4'], index=df_index.columns)
loadings_matrix

In [ ]:
num = 3
weight = []
for i in range(0, 4):
    weight.append((loadings[i][:num]*explained_variance_ratios[:num]).sum()/(explained_variance_ratios[:num].sum()))
weight = np.array(weight)
norm_weight = weight/(weight.sum())
pd.DataFrame({'weight':norm_weight}, index=df_index.columns)

In [ ]:
df_merge['udi19w'] = df_index.apply(lambda x: np.power(x, norm_weight).prod(), axis=1)
norm01(df_merge, 'udi19w')

In [ ]:
geo_file_root = './geo_file'
if not os.path.exists(geo_file_root):
    os.mkdir(geo_file_root)

In [ ]:
gpd.GeoDataFrame(df_merge).to_file(os.path.join(geo_file_root, '20230806point.shp'), driver='ESRI Shapefile',encoding='utf-8')

In [ ]:
df_merge_udi = df_merge.loc[:,['eFUA_ID', 'eFUA_name', 'Cntry_ISO', 'Cntry_name', 'Regions', 'udi01n', 'udi01wn', 'udi19n', 'udi19wn']]
df_merge_udi_mean = df_merge_udi.groupby(['eFUA_ID', 'eFUA_name', 'Cntry_ISO', 'Cntry_name', 'Regions']).mean().reset_index()
df_merge_udi_sum = df_merge_udi.groupby(['eFUA_ID', 'eFUA_name', 'Cntry_ISO', 'Cntry_name', 'Regions']).sum().reset_index()

In [ ]:
df_merge_mean_sum = pd.merge(df_merge_udi_mean.iloc[:,[0,-4,-3,-2,-1]].rename(columns={'udi01n':"udi01_m",'udi01wn':'udi01w_m','udi19n':'udi19_m','udi19wn':'udi19w_m'}),
                             df_merge_udi_sum.iloc[:,[0,-4,-3,-2,-1]].rename(columns={'udi01n':"udi01_s",'udi01wn':'udi01w_s','udi19n':'udi19_s','udi19wn':'udi19w_s'}))

In [ ]:
df_merge.loc[df_merge['Cntry_name'].isin(['Taiwan', 'HongKong', 'Macao']),'Cntry_ISO'] = 'CHN'
df_merge.loc[df_merge['Cntry_name'].isin(['Taiwan', 'HongKong', 'Macao']),'Cntry_name'] = 'China'

In [ ]:
import copy
df_merge_copy = copy.deepcopy(df_merge)
df_merge_copy = df_merge_copy.loc[df_merge['grid_code']>=21, :]
df_merge_copy['count'] = df_merge_copy.groupby('eFUA_ID')['Id'].transform(lambda x: x.count())

In [ ]:
df_merge_country = df_merge_copy.loc[(df_merge_copy['count']>10), ['Cntry_ISO','Cntry_name','Regions','udi19wn']]

In [ ]:
def in_qrange(ser, q):
    return ser.between(*ser.quantile(q=q))

In [ ]:
df_merge_country_p99 = df_merge_country.loc[df_merge_country.groupby(['Cntry_ISO','Cntry_name','Regions'])['udi19wn'].transform(in_qrange, q=[0.01, 0.99])]

In [ ]:
df_merge_country_p99_std = df_merge_country_p99.loc[:,['Cntry_ISO','Cntry_name','Regions','udi19wn']].groupby(['Cntry_ISO','Cntry_name','Regions']).std()
df_merge_country_p99_mean = df_merge_country_p99.loc[:,['Cntry_ISO','Cntry_name','Regions','udi19wn']].groupby(['Cntry_ISO','Cntry_name','Regions']).mean()
df_merge_country_p99_sum = df_merge_country_p99.loc[:,['Cntry_ISO','Cntry_name','Regions','udi19wn']].groupby(['Cntry_ISO','Cntry_name','Regions']).sum()
df_merge_country_p99_inq = df_merge_country_p99_std/(np.sqrt(df_merge_country_p99_mean*(1-df_merge_country_p99_mean)))
def get_gini(data):
    return gini(np.array(data))
df_merge_country_p99_gini =  df_merge_country_p99.loc[:,['Cntry_ISO','Cntry_name','Regions','udi19wn']].groupby(['Cntry_ISO','Cntry_name','Regions']).aggregate({'udi19wn':get_gini})
## to csv gini
df_merge_country_p99_sum.rename(columns = {'udi19wn':'sum'}).join(df_merge_country_p99_mean.rename(columns = {'udi19wn':'mean'})).join(df_merge_country_p99_gini.rename(columns = {'udi19wn':'gini'})).sort_values('gini').to_csv('20230806udi_country.csv')

In [ ]:
df_merge_country_p99_sum_pre = df_merge_country_p99_sum.reset_index().sort_values('udi19wn', ascending=False)
df_merge_country_p99_sum_pre['udi19wn%'] = df_merge_country_p99_sum_pre['udi19wn']/(df_merge_country_p99_sum_pre['udi19wn'].sum())
df_merge_country_p99_sum_pre.iloc[:10,:]

In [ ]:
df_merge_copy.loc[(df_merge_copy['count']>10),['built01', 'pop_log01','Length_total01', 'ntl_log01', 'udi19w']].rename(columns={'built01':'Built',
                                                                                                                                'pop_log01':'Pop','Length_total01':'Roads',
                                                                                                                                'ntl_log01':'NTL','udi19w':'UDI'}).to_csv('20230816corrplot.csv', index=False)

In [ ]:
import libpysal
import splot
import pysal
from esda.moran import Moran_Local_BV
from splot.esda import moran_scatterplot
from libpysal.weights.spatial_lag import lag_spatial

In [ ]:
y = fua_mean_gini['gini'].values
x = fua_mean_gini['mean'].values
w = libpysal.weights.KNN.from_dataframe(fua_mean_gini, k=10, ids='eFUA_ID')
# w = libpysal.weights.DistanceBand.from_dataframe(fua_mean_gini, threshold=100000, binary=False, alpha=-2.)
moran_loc_bv = Moran_Local_BV(x, y, w)
fig, ax = moran_scatterplot(moran_loc_bv, p=0.05)
ax.set_xlabel('Inequality')
ax.set_ylabel('Spatially Lag of inequality')
plt.show()

In [ ]:
from libpysal.weights.spatial_lag import lag_spatial
lag = lag_spatial(moran_loc_bv.w, moran_loc_bv.zy)
# x = moran_loc_bv.zx

In [ ]:
fua_mean_gini_moran = pd.DataFrame({'x_Is':moran_loc_bv.zx,
                                    'y_Lag':lag_spatial(moran_loc_bv.w, moran_loc_bv.zy),
                                    'p_sim':moran_loc_bv.p_sim,
                                    'label':moran_loc_bv.q})